In [1]:
library(eurostat)
library(ggplot2)
library(corrplot)
library(vtable)
library(cowplot)
library(moments)
library(e1071)

options(width=160, repr.plot.width=11, repr.plot.height=8)     

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
corrplot 0.92 loaded

Loading required package: kableExtra


Attaching package: ‘e1071’


The following objects are masked from ‘package:moments’:

    kurtosis, moment, skewness




# Úkoly

V tomto úkolu provedete analýzu datasetu z Eurostatu. Přístup k datům umožňuje balík [eurostat](https://ropengov.github.io/eurostat/articles/eurostat_tutorial.html). Počátečními kroky vás skript provede, následně bude práce na vás.

Vybereme si např. data, která obsahují slovo "heating":

In [2]:
search_eurostat(pattern='heating')

title,code,type,last update of data,last table structure change,data start,data end,values
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cooling and heating degree days by NUTS 3 regions - annual data,nrg_chddr2_a,dataset,01.02.2023,01.02.2023,1979,2022,NA
Cooling and heating degree days by NUTS 3 regions - monthly data,nrg_chddr2_m,dataset,01.02.2023,01.02.2023,1979M01,2022M12,NA
Use of renewables for heating and cooling - details,nrg_ind_urhcd,dataset,27.01.2023,27.01.2023,2004,2020,NA
Cooling and heating degree days by country - annual data,nrg_chdd_a,dataset,01.02.2023,01.02.2023,1979,2022,NA
Cooling and heating degree days by country - monthly data,nrg_chdd_m,dataset,01.02.2023,01.02.2023,1979M01,2022M12,NA
Cooling and heating degree days by NUTS 3 regions - annual data,nrg_chddr2_a,dataset,01.02.2023,01.02.2023,1979,2022,NA
Cooling and heating degree days by NUTS 3 regions - monthly data,nrg_chddr2_m,dataset,01.02.2023,01.02.2023,1979M01,2022M12,NA
Cooling and heating degree days by NUTS 3 regions - monthly data,nrg_chddr2_m,dataset,01.02.2023,01.02.2023,1979M01,2022M12,NA
Cooling and heating degree days by NUTS 3 regions - annual data,nrg_chddr2_a,dataset,01.02.2023,01.02.2023,1979,2022,NA


Dostali jsme tabulku, v níž je přehled datových sad s klíčovým slovem, plus pár údajů. Pro stažení datasetu je potřeba jeho identifikátor - `code`. Stáhneme si vybraný dataset:

In [3]:
id <- 'nrg_chddr2_a'
dat <- get_eurostat(id=id)
head(dat)

Table nrg_chddr2_a cached at /tmp/RtmpxxhXS7/eurostat/nrg_chddr2_a_date_code_FF.rds



unit,indic_nrg,geo,time,values
<chr>,<chr>,<chr>,<date>,<dbl>
NR,CDD,AT,2022-01-01,29.40
NR,CDD,AT1,2022-01-01,62.78
NR,CDD,AT11,2022-01-01,94.55
NR,CDD,AT111,2022-01-01,80.73
NR,CDD,AT112,2022-01-01,126.13
NR,CDD,AT113,2022-01-01,62.72


[Dataset](https://en.wikipedia.org/wiki/Heating_degree_day) obsahuje měřící jednotku (NR, numeric), indikátor ([CDD, HDD](https://en.wikipedia.org/wiki/Heating_degree_day)), geolokaci podle [NUTS](https://cs.wikipedia.org/wiki/NUTS), den získání dat a hodnotu.

Vybereme si nějaký z krajů [ČR](https://cs.wikipedia.org/wiki/CZ-NUTS), dále třeba [Finska](https://cs.wikipedia.org/wiki/FI-NUTS), [Norska](https://cs.wikipedia.org/wiki/NO-NUTS) a [Španělska](https://cs.wikipedia.org/wiki/ES-NUTS):

In [4]:
NUTS3 <- c('FI1D3', 'CZ031', 'NO011', 'ES230')
data <- dat[dat$geo %in% NUTS3,]
head(data)

unit,indic_nrg,geo,time,values
<chr>,<chr>,<chr>,<date>,<dbl>
NR,CDD,CZ031,2022-01-01,13.64
NR,CDD,ES230,2022-01-01,182.98
NR,CDD,FI1D3,2022-01-01,1.55
NR,CDD,NO011,2022-01-01,0.00
NR,HDD,CZ031,2022-01-01,3085.93
NR,HDD,ES230,2022-01-01,1984.78


Balík [eurostat](https://ropengov.github.io/eurostat/articles/eurostat_tutorial.html) umožňuje i labelování dat. To je praktické pro přehled, méně už pro práci.

In [5]:
data_labels <- label_eurostat(data)
head(data_labels)

unit,indic_nrg,geo,time,values
<chr>,<chr>,<chr>,<date>,<dbl>
Number,Cooling degree days,Jihocecký kraj,2022-01-01,13.64
Number,Cooling degree days,La Rioja,2022-01-01,182.98
Number,Cooling degree days,Pohjois-Karjala,2022-01-01,1.55
Number,Cooling degree days,Oslo (statistical region 2016),2022-01-01,0.00
Number,Heating degree days,Jihocecký kraj,2022-01-01,3085.93
Number,Heating degree days,La Rioja,2022-01-01,1984.78


## Úkol 1: předefinujte sloupce:
- `data$geo` na typ factor,
- `data$time` na typ Date.

Nápověda: `as.factor()`, `as.Date()`

In [6]:
#=== CODE HERE ===#

### Úkol 2

Upravte data (nebo vytvořte novou tabulku data) tak, by obsahovala sloupce `geo`, `time`, `cdd_vals` a `hdd_vals` s hodnotami CDD a HDD.

Nápověda: možností je více, můžete vytvořit dvě tabulky zvlášť pro CDD a pro HDD, smazat zbytečné sloupce (nastavit jim hodnotu NULL) a pak je spojit pomocí `merge()`.

In [7]:
#=== CODE HERE ===#

<font color="blue">**Nyní zpět k výkladu.**</font>

---

### Úkol 3

Vykreslete bodový graf (scatter plot) a diskutujte o vlastnostech dat. Nápověda: `pairs()`.

In [8]:
#=== CODE HERE ===#

<font color="blue">**Nyní zpět k výkladu.**</font>

---

### Úkol 4.a: základní popisné statistiky

Věnujte se popisné statistice. Zjistěte nejdůležitější statistické míry a interpretujte je. Nápověda: `summary()`.

In [9]:
#=== CODE HERE ===#

Toto nám nemusí úplně vyhovovat. Proč? 

### Úkol 4.b: základní popisné statistiky (pokr.)

Zkusíme se tedy zaměřit na detaily. Uděláme summary pro jednu z oblastí.

In [10]:
#=== CODE HERE ===#

Ještě lepší může být použít `subset()`.

In [11]:
#=== CODE HERE ===#

### Úkol 4.c: základní popisné statistiky (pokr.)

Poslední varianta je použít `sumtable()`.

In [12]:
#=== CODE HERE ===#

<font color="blue">**Nyní zpět k výkladu.**</font>

---

### Úkol 5.a: histogramy

Vykreslete histogramy a diskutujte nad výsledky.

In [13]:
#=== CODE HERE ===#

<font color="blue">**Nyní zpět k výkladu.**</font>

---

### Úkol 5.b: box plot

Vykreslete boxploty a diskutujte nad výsledky. Co obsahuje boxplot?

In [14]:
#=== CODE HERE ===#

<font color="blue">**Nyní zpět k výkladu.**</font>

---

### Úkol 5.c: houslový graf

Vykreslete houslový graf (violin plot) a interpretujte výsledky.

In [15]:
#=== CODE HERE ===#

<font color="blue">**Nyní zpět k výkladu.**</font>

---

### Úkol 6.a: data CZ031

Podívejme se detailněji na data CZ031. Vykreslete v jednom obrázku histogram, KDE odhad hustoty a pro porovnání hustotu normálního rozdělení se střední hodnotou a rozptylem shodným s výběrovými statistikami dat. Diskutujte o výsledcích.
Nápověda: `geom_histogram()`, `geom_density()`, `stat_function()`.

In [16]:
#=== CODE HERE ===#

### Úkol 6.b: data CZ031 (pokr.)

Proveďte analýzu normality pomocí Q-Q plotu. Nápověda: `stat_qq()`, `stat_qq_line()`.

In [17]:
#=== CODE HERE ===#

### Úkol 6.c: data CZ031 (pokr.)

Určete kvantily téhož datasetu a interpretujte opět Q-Q plot.

In [18]:
#=== CODE HERE ===#

### Úkol 6.d: data CZ031 (pokr.)

Otestujte normalitu pomocí Shapirova-Wilkova testu. Příslušnou funkci zkuste najít sami, interpretujte výsledky.

In [19]:
#=== CODE HERE ===#

<font color="blue">**Nyní zpět k výkladu.**</font>

---

### Úkol 7.a: Korelační analýza

Proveďte korelační analýzu. Určete hodnoty korelačních koeficientů a interpretujte je. Použijte funkci `ggpairs()` z balíčku `GGally()`.

In [20]:
#=== CODE HERE ===#

### Úkol 7.b: korelační analýzy (pokr.)

Vykreslete grafy pomocí `corrplot()`. Interpretujte výsledky a prozkoumejte různá nastavení parametru `method`.

In [21]:
#=== CODE HERE ===#